In [6]:
# -*- coding:utf-8 -*-
import happybase
import pymongo
import time
from bson import ObjectId

def get_mongo_db():
    conn = pymongo.MongoClient('192.168.1.6',27017).phone
    conn.authenticate('phonedata','phonedata1805')
    return conn


def get_hbase_db():
    conn = happybase.Connection('47.93.144.37')
    return conn


# 添加导出日志
def set_export2hbase_log(src_coll_name, minid, maxid):
    db = get_mongo_db()
    strtime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    doc = {"src_coll_name": src_coll_name,
          "minid":minid,
          "maxid":maxid,
          "time":strtime}
    db.result_export2hbase_log.insert_one(doc)

    
# 获取本次导出的起始'_id'
def get_start_id(src_coll_name):
    db = get_mongo_db()
    maxid = ObjectId('000000000000000000000000')    
    rslt = db.result_export2hbase_log.find({"src_coll_name": src_coll_name}).sort("_id", pymongo.DESCENDING)
    for rs in rslt:
        maxid = rs["maxid"]
        print('from log, maxid:', maxid)
        break
    print(src_coll_name, ' maxid:', maxid)
    return maxid    
    

# 生成表时调用一次，其它时候请谨慎调用
# ！！！请谨慎调用
def create_hbase_tables():
    conn = get_hbase_db()    
    tables = conn.tables()
    print("the tables before creating: ", tables)       
    
    # 创建phonepraise_in_phoneinfo
    phoneinfo_families = {
        'needful': dict(),
        'option': dict()
    }
    if b'phonepraise_in_phoneinfo' in tables:
        conn.disable_table(b'phonepraise_in_phoneinfo')
        conn.delete_table(b'phonepraise_in_phoneinfo')
    conn.create_table(b'phonepraise_in_phoneinfo', phoneinfo_families)
    
    # 创建phonepraise_in_comments
    comments_families = {
        'needful': dict(),
        'option': dict()
    }
    if b'phonepraise_in_comments' in tables:
        conn.disable_table(b'phonepraise_in_comments')
        conn.delete_table(b'phonepraise_in_comments')
    conn.create_table(b'phonepraise_in_comments', comments_families)
    
    tables = conn.tables()
    print("the tables after created: ", tables)
    conn.close()


# 清空所有导出结果和log，请谨慎调用
# ！！！请谨慎调用
def clean_all_exported():
    mgdb = get_mongo_db()
    mgdb.result_export2hbase_log.delete_many({'src_coll_name': 'result_phoneinfo'})
    mgdb.result_export2hbase_log.delete_many({'src_coll_name': 'result_comment'})    
    create_hbase_tables()

# 把手机信息表导入到hbase
def export_phoneinfo2hbase():
    print('exporting phoneinfo to hbase ...')
    mgdb = get_mongo_db()
    srctbl = mgdb['result_phoneinfo']
    minid = get_start_id('result_phoneinfo')
    maxid = minid
    cursor = srctbl.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)
    
    hbdb = get_hbase_db()
    dsttbl = hbdb.table('phonepraise_in_phoneinfo')
    
    last_row_key = '0000000000000000'
    for key, data in dsttbl.scan(limit=10, reverse=True):
        print(key, data) 
        last_row_key = key
        break
    index = int(last_row_key)
    
    with dsttbl.batch(batch_size=1024) as dstbatch:
        for cur in cursor:
            phoneid = '%d' % cur.get('phoneid')
            _id = str(cur.get('_id'))         
            dstdoc = {
                'needful:phoneid': phoneid,
                'needful:phonename': cur.get('phonename'),
                'needful:keyword': cur.get('keyword'),
                'option:_id': _id,
                'option:code': cur.get('code'),
                'option:add_time': cur.get('add_time')
            }
            index += 1
            dstbatch.put('%016d' % index, dstdoc)
            maxid = cur.get('_id')
        dstbatch.send()
    
    set_export2hbase_log('result_phoneinfo', minid, maxid)
    hbdb.close()
    print('exported phoneinfo to hbase end')

# 把评论表导入到hbase
def export_comment2hbase():
    print('exporting comment to hbase ...')
    mgdb = get_mongo_db()
    srctbl = mgdb['result_comment']
    minid = get_start_id('result_comment')
    maxid = minid
    cursor = srctbl.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)
    
    hbdb = get_hbase_db()
    dsttbl = hbdb.table('phonepraise_in_comments')
    
    last_row_key = '0000000000000000'
    for key, data in dsttbl.scan(limit=10, reverse=True):
        print(key, data) 
        last_row_key = key
        break
    index = int(last_row_key)
    
    with dsttbl.batch(batch_size=1024) as dstbatch:
        for cur in cursor:
            phoneid = cur.get('phoneid')
            _id = str(cur.get('_id'))        
            dstdoc = {
                'needful:phoneid': phoneid,
                'needful:comment_words': cur.get('comment_words'),
                'needful:comment_score': cur.get('comment_score'),
                'needful:comment_date': cur.get('comment_date'),
                'needful:proid': cur.get('proid'),
                'needful:from': cur.get('from'),
                'needful:userid': cur.get('userid'),
                'needful:userlevel': cur.get('userlevel'),
                'needful:province': cur.get('province'),
                'needful:terminal': cur.get('terminal'),
                'needful:_id': _id
            }
            index += 1
            dstbatch.put('%016d' % index, dstdoc)
            maxid = cur.get('_id')
        dstbatch.send()
    
    set_export2hbase_log('result_comment', minid, maxid)
    hbdb.close()
    print('exported comment to hbase end')

def main():
    export_phoneinfo2hbase()
    export_comment2hbase()

# 打印少量数据，和部分统计结果    
def test():
    print('===========================print test info===================================')
    hbdb = get_hbase_db()
    #查看所有表
    tables = hbdb.tables()
    print(tables)
    # 查看 comments 表
    tbl = hbdb.table('phonepraise_in_comments')
    index = 0
    for key, data in tbl.scan():
        index += 1
        if index % 1000 == 1:
            print(key, data.get(b'needful:phoneid').decode('utf-8'), data.get(b'needful:comment_words').decode('utf-8'))
    print('comments total records: ', index)    
    # 查看 phoneinfo 表
    tbl = hbdb.table('phonepraise_in_phoneinfo')
    index = 0
    for key, data in tbl.scan():
        index += 1
        if index % 1000 == 1:
            print(key, data.get(b'needful:phoneid').decode('utf-8'), data.get(b'needful:phonename').decode('utf-8'))
    print('phoneinfo total records: ', index)      
    hbdb.close()
    
if __name__ == '__main__':
    main()
    test()

exporting phoneinfo to hbase ...
from log, maxid: 5966e981e138230ea3d52d2a
result_phoneinfo  maxid: 5966e981e138230ea3d52d2a
b'0000000000002144' {b'needful:keyword': b'\xe4\xb8\xad\xe5\x85\xb4-G720', b'needful:phoneid': b'2389', b'needful:phonename': b'\xe4\xb8\xad\xe5\x85\xb4 G720', b'option:_id': b'5966e981e138230ea3d52d2a', b'option:add_time': b'2017-07-13 11:31:13', b'option:code': b'ZXG720'}
exported phoneinfo to hbase end
exporting comment to hbase ...
from log, maxid: 595a2080cf1e8c0eb82f18ba
result_comment  maxid: 595a2080cf1e8c0eb82f18ba
b'0000000002535421' {b'needful:_id': b'595a2080cf1e8c0eb82f18ba', b'needful:comment_date': b'2016-06-03 00:37', b'needful:comment_score': b'\xe5\xa5\xbd\xe8\xaf\x84', b'needful:comment_words': b'\xe5\xa5\xbd\xe8\xaf\x84', b'needful:from': b'zte', b'needful:phoneid': b'388', b'needful:proid': b'1745', b'needful:province': b'', b'needful:terminal': b'', b'needful:userid': b'148****0455', b'needful:userlevel': b''}
exported comment to hbase end
=

b'0000000000159001' 22 新机到手，速度还可以，就是送的钢化膜有点小！
b'0000000000160001' 11 发货速度很快，质量不错，使用时说明书没有
b'0000000000161001' 112 电话没有耳机不怎么好
b'0000000000162001' 150 商品收到，感觉不错！
b'0000000000163001' 11 手机是没什么问题。不过这物流真是无力吐槽，这物流分还那么高不知道哪里来的。。。
b'0000000000164001' 54 很不错的手机，现在的价格很便宜了
b'0000000000165001' 43 非常满意！
b'0000000000166001' 75 我其实是图好评返30元话费来的，希望不要食言。不过手机确实不错，十核加4G运行内存，反应超快。唯一美中不足的是后摄像头突出太多，怕磨坏。
b'0000000000167001' 22 急也来咯来咯
b'0000000000168001' 31 手机用起来比较顺手，上网基本上不会断，系统用时间长了有时候会略卡。
b'0000000000169001' 1013 手机真不错，下午用了用，就是轻快！
b'0000000000170001' 22 手机不错反应挺快的。一直信赖京东
b'0000000000171001' 151 东西很给力，老婆很喜欢！
b'0000000000172001' 156 手机不错，性价比高
b'0000000000173001' 104 手机挺好用的，手感不错。就是送的钢化膜小了点，总体感觉还是挺满意的。
b'0000000000174001' 75 手机好，我喜欢。发货慢了点
b'0000000000175001' 117 宝贝收到了，用着刚刚好，比实体店便宜多了，质量还不错，喜欢的就不要犹豫了！?
b'0000000000176001' 48 手机运行很好，
b'0000000000177001' 18 宝贝不错，爸妈挺喜欢的
b'0000000000178001' 12 用了非常不错！质量非常有保障！
b'0000000000179001' 100 不错
b'0000000000180001' 59 非常快 各种喜欢
b'0000000000181001' 54 和官网买的差不多，没什么区别，但速度不太像京东的，这个不太

b'0000000000341001' 428 看着，不错，电池没电，2013年产的，这都17年啦，这手机，真不想过多的评价！还没开机，不知道运行怎么样。只要支持nfc就够了！不敢奢求太多！
b'0000000000342001' 417 给老丈母娘买的手机收到了，用了几感觉还是可以滴！运行挺好，就是打电话时候感觉耳边有很大的电流声音！
b'0000000000343001' 76 机器很不错?、速度很快，机器屏幕也好
b'0000000000344001' 282 便宜实惠，用的挺好，
b'0000000000345001' 462 好噢周一休息太多的痘痘自言自语自言自语夕阳西下好了
b'0000000000346001' 463 手机挺好的，就是比想象用的笨重一点。
b'0000000000347001' 211 水果断电话费用的话费用的话费！！！
b'0000000000348001' 437 手机很好用，一下买了三个，智能手机啊
b'0000000000349001' 212 比较满意，性价比很高
b'0000000000350001' 187 不错，通话声音很清楚
b'0000000000351001' 182 整体还行，就是解锁开关使用体验不好
b'0000000000352001' 224 暂时用着都还好，老年人手速慢，便宜好用
b'0000000000353001' 225 买给婆婆用的，挺不错的，声音够大声，大小也合适
b'0000000000354001' 452 还可以，没太多说的额..
b'0000000000355001' 241 手机各方面都不错，比较适合老年人，这次我一下买了两部
b'0000000000356001' 451 老人家用，设置老人界面很好用，操作容易，一目了然！
b'0000000000357001' 455 我是买来送我妈妈的，这才一个月都没用到手机啥声音都没的，看电视听音乐都没的声音，问客服甩都不甩，差评，我就当自己吸取的经验，下次别买杂牌手机
b'0000000000358001' 457 东东都还是可以，老人使用方便
b'0000000000359001' 224 还行吧！
b'0000000000360001' 461 给爸妈用的，外观还不错
b'0000000000361001' 464 中学生用，功能少，免

b'0000000000524001' 26 给舅舅舅妈用，操作起来很流畅，老人很开心。
b'0000000000525001' 112 挺好的！。。。。。。。。。。。。
b'0000000000526001' 168 我觉得，很好！没有买错，还有护眼功能，特别适合学生使用，运行流畅，下次再光顾。屏幕也挺灵敏的，就是安卓版本有点低，拿在手上很气派。还有前置柔光灯，拍照还有相似的名人，测脸龄，这些都很有趣。支持360希望越来越好。
b'0000000000527001' 5 第二次买这手机了，性价比很高，不过比上次贵了一百。
b'0000000000528001' 27 ???
b'0000000000529001' 18 给我妈买的手机，本来是准备买水果的，但是她非说不要，自己选了这个小米的，用起来还挺流畅的
b'0000000000530001' 151 电掉的好快，正常使用9个小时就关机了，做备机的小心了
b'0000000000531001' 2 非常满意，用着非常完美
b'0000000000532001' 7 好，就是物流比较慢
b'0000000000533001' 18 运行很流畅，速度蛮凉快的，不错
b'0000000000534001' 85 不知道?，就买给老人用，还没问过，只听说机型小。
b'0000000000535001' 7 内存那个是什么鬼

16G的内存，可用容量7G都不到
b'0000000000536001' 5 出去的中央祝
b'0000000000537001' 7 新买的手机，为什么黑屏什么都没有？
b'0000000000538001' 18 一直都喜欢来京东买东西，服务、质量一直都很给力！
b'0000000000539001' 31 可是你深深你说你是
b'0000000000540001' 10 非常满意，用了一个多月才来评价！！！！
b'0000000000541001' 7 全网通的，支持移动，电信，联通234G，看起来很不错的，配置4+64的这价格便宜了
b'0000000000542001' 169 相当的不错，第二个乐视手机，乐视挺起来
b'0000000000543001' 29 差强人意吧！！！！！！！！！性价比确实高，但是使用的感觉不太好，发热比较严重
b'0000000000544001' 30 刚

b'0000000000694001' 64 好！！！！！！！！！！
b'0000000000695001' 156 还可以，
b'0000000000696001' 91 很漂亮 特别好看 喜欢的不得了
b'0000000000697001' 92 好好好好好好
b'0000000000698001' 22 手机挺不错的，正版机子，质量挺好的 ，下次需要手机还来他家，大家有需要的可以来买
b'0000000000699001' 9 手机真心的不错
b'0000000000700001' 98 非常实用，老人家上手很快。电量大，可以待机一个多星期。
b'0000000000701001' 48 有的时候会微信视频没有声音，可能价格决定他就这个水平吧，一分钱一分货
b'0000000000702001' 56 刚收到，应该正品
b'0000000000703001' 95 很好
b'0000000000704001' 96 怎么什么都不送
一个手机壳，钢化膜都没，耳机都没有
b'0000000000705001' 13 手机不错，好评^&omega;^
b'0000000000706001' 27 不错&hellip;
b'0000000000707001' 5 感觉还好，买给爸爸的，他说不卡
b'0000000000708001' 26 态度差，物流居然送错地方。
b'0000000000709001' 75 手机不错，不过比官网的要慢一些，总体不错，好评
b'0000000000710001' 24 价格比其它地方便宜，用起来目前还不错，就是中途物流信息看不懂，未实时更新
b'0000000000711001' 99 真心不错的好手机，就是京东快递有点慢。
b'0000000000712001' 9 实物和图片没区别
b'0000000000713001' 64 不错
b'0000000000714001' 43 好
b'0000000000715001' 100 用起来还不错0000
b'0000000000716001' 48 给表格买的手机。据说不错，。
b'0000000000717001' 821 电池一整天够用
b'0000000000718001' 118 手机的手电筒亮不了！差评！
b'0000000000719001' 85 不错，不错
b'00

b'0000000000877001' 11 系统流畅性能好，相当有手感好
b'0000000000878001' 11 期待已久的苹果终于到手了，是正品，好喜欢，快递小哥很给力，手机棒棒哒
b'0000000000879001' 12 非常棒的购物体验 物流很及时 货物是正品
b'0000000000880001' 12 不错。 6.18送货也很及时。京东小哥也不错。随机减免活动力度再大点就好了
b'0000000000881001' 7 手机反应速度快，价格便宜，照相也很清楚，玩游戏也不卡，外观漂亮，物美价廉，在京东买贵重物品放心，快还有保证，下次还会再来购买，给五星好评加分享，支持国货，没得说，小米手机性价比很高，满意加超赞！！！
b'0000000000882001' 17 刚打开手机被惊艳到了。特别棒，手感很好，特别好看。下了个鲁大师测试了下。排行第一噢哈哈。而且一等奖。加上2550入手的高配。简直不能更满意了哈哈???PS：拍得不好百度来凑哈哈。还有。不要升级系统，升级后耗电快了，手机充电也是很快，就是有点热，不过戴套没感觉了，表示并没有评论说的wifi不稳定的问题，应该是路由器的问题我觉得，移动网络也很稳的，而且可以加SD卡，很不错的扩展哈哈。反正就是不错。??支持国货咯嘿嘿。赞赞赞
b'0000000000883001' 22 618买的，超级划算哈，比正常价还要低好多的，点32个赞..
b'0000000000884001' 24 我公民你自尊心老婆哦破哦婆婆哦破婆婆哦婆婆
b'0000000000885001' 7 好&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;&hellip;
b'0000000000886001' 25 好
b'0000000000887001' 36 还行吧，就是有点轻，手感不是太好。
b'00000000008

b'0000000001033001' 8 非常好用，我很满意，价格给力没话说。
b'0000000001034001' 10 呵呵。商品这么快就到了，还不错哦，下次来你可要优惠哦^_^  
网上购物这么激烈，没想到店家的服务这么好，商品质量好而价低廉，我太感谢你了！
国美很讲信誉，而且很不错哦，国美买东东，我很满意~
期盼之中，终于等到了心爱的东东，谢谢！
b'0000000001035001' 22 一模一样，手感非常好
b'0000000001036001' 145 电子发票在哪儿啊？配送快，优惠给力，东西还没用
b'0000000001037001' 23 可以
b'0000000001038001' 33 非常好，速度很快
b'0000000001039001' 423 很给力，摄像头很好，金属机身，速度流畅，好满意啊，国美配送速度又快，超级喜欢呢
b'0000000001040001' 42 户未及时评价，显示为默认评价。
b'0000000001041001' 107 刚买回来就用了，处理速度很快，不错，赞一个。
b'0000000001042001' 9 包装看起来很好，包得很用心，相信货一定很好，谢谢了！
b'0000000001043001' 124 服务态度好，手机性能不错
b'0000000001044001' 20 特意用过之后来评价，手机一点也不卡，拍照也好清晰，外观设计漂亮，几百块就买到性价比很高的手机，超满意，我身边好多人都用华为手机，相信质量，wf值得购买，手机一点也不卡，拍照也好清晰，外观设计漂亮，几百块就买到性价比很高的手机，超满意，我身边好多人都用华为手机，相信质量。华为手机国货精品。
b'0000000001045001' 26 还可以  我已经买了好几部华为了 支持国产
b'0000000001046001' 10 物美价廉
b'0000000001047001' 25 我想要发票，可不知道怎么操作。快递到新疆只用三天，快递公司负责，伞落在店里打电话让去拿，五分。不满意的是没有客服不知道发票怎么搞。
b'0000000001048001' 56 手机漂亮，用起来顺畅，喜欢
b'0000000001049001' 37 之前买荣耀8现在买华为P10拍照技术杠杠的  5.0   6666的，做工精细。好好好
b'0000000

b'0000000001191001' 9 快递慢的像蜗牛，是真的垃圾
b'0000000001192001' 9 第一次买国产，试试的心态，选机子还蛮纠结~快递超快~打开以后非常非常喜欢~颜值蛮高的手感也很好~~觉得除了系统不一样，其他方面不输苹果、系统用习惯就会好的~很满意，特意来写评论给大家参考，决定以后都转粉儿这个牌子了。很满意很喜欢。明天开用，过阵子再来追加评论~
b'0000000001193001' 9 好
b'0000000001194001' 9 模样漂亮极了，反应快，妥妥滴！
b'0000000001195001' 9 自己用的小6给我哥买的小8，值得购买
b'0000000001196001' 9 我都想把我的mate8换了  看着太舒服了  我这个有点大 希望以后出很好的  然后我到时候换  呵呵
b'0000000001197001' 9 好手机 华为造
b'0000000001198001' 9 东西很好，给力
b'0000000001199001' 9 真心不赖，支持一下国产
b'0000000001200001' 9 为什么我的前置摄像头有点偏呢？
b'0000000001201001' 9 手感不错
b'0000000001202001' 9 性价比太高了
b'0000000001203001' 9 还可以，挺不错呢
b'0000000001204001' 9 就是要剪卡，卡大了放不进手机……
b'0000000001205001' 9 不错，一如既往，支持国货，第二部华为
b'0000000001206001' 9 实物跟网上的参考图相差甚远
b'0000000001207001' 9 之前从3C到荣耀7一直都用的很不错，本来抱着满怀期待的荣耀8到手后，却是后悔莫及！金玉其外，败絮其中，性能全不及荣耀7，微信卡，游戏卡，浏览网页也卡，真想砸了！大家千万不要被外观蒙蔽了，买之前一定关注论坛！切记！！！
b'0000000001208001' 9 手机的确很好～前天早晨订购今天早上就拿到上手了～系统好像没有小米系统好用～国产手机用了六七款～就小米和华为给我感觉很好！乐视手机最垃圾
b'0000000001209001' 9 外表很炫酷～反应野很快，唯一的缺点就是屏幕没有合适的贴膜
b'0000000001210001' 9 暂时没毛病

b'0000000001341001' 13 感觉不错，期待华为继续荣耀
b'0000000001342001' 13 很久以前其实就在关注华为的这款手机在第一次发售第一次发售的时候犹豫了一下然后错过了然后第二次发售的时候又没抢上唉总之这个爱他的过程很艰辛呢但还是耐不住他双镜头的吸引在的好多次发售之后我终于在千万人之中抢到了他激动之情溢于言表啊      嗯在烧心的等待中，我终于迎来了它。不负我望，它的颜值首先征服了我，然后是屏幕的灵敏和溜冰般的享受      试过了相机，很专业，很合心意。特意用了两天才来评价，发热问题还可以，应该和天气有很大的关系吧，这两天简直可以烤串了。续航真是赞  ，闪充也很牛，大概两个多小时就可以充满。这里特别要说，VR眼镜，飞一般的感觉啊。看的仲基      。荣耀V8 低调奢华有内涵赞
b'0000000001343001' 13 两天才到  没贴膜  像素可以  有点发热总体还不错
b'0000000001344001' 13 宝贝直得拥有，发货很快物流也快。
b'0000000001345001' 13 不错 物流挺快的   手机很流畅清晰
b'0000000001346001' 13 华为荣耀V8，抢了3天才抢到，昨天刚到手，铂光金全网通4G十32G，细节非常到位，还具有NFC功能，指纹锁双镜头，反应快，外观内在非常精致，喜欢V8，支持华为
b'0000000001347001' 13 华为非常不错，，一直支持！
b'0000000001348001' 13 嘻嘻，同事看我用觉得好，给帮忙买的，都给点赞
b'0000000001349001' 13 给老爸买的，脱离诺记之后一直用的华为，这款华为V8屏够大，清晰度很高，操作简单。智灵键放在侧面，用着很方便，反馈速度快。双摄像头看着很高端，虽然拍照功能很少使用。。  总之是很满意的。
b'0000000001350001' 13 v8买一个真难啊 ，抢不到 抢不到，真大气，精致。
b'0000000001351001' 13 自从我自己用上后，朋友都让帮着买，唉，以后有事做了。只怪手机太给力。好评。
b'0000000001352001' 13 这款手机确实很不错，外观很漂亮，速度很快，性价比高，手机刚拿到手确实很惊喜，唯一不好的是充电插孔跟通用的不兼容，不过充电速度确实超快。正式从

IOError: IOError(message=b'org.apache.hadoop.hbase.DoNotRetryIOException: hconnection-0x5163057f closed\n\tat org.apache.hadoop.hbase.client.ConnectionManager$HConnectionImplementation.locateRegion(ConnectionManager.java:1174)\n\tat org.apache.hadoop.hbase.client.RpcRetryingCallerWithReadReplicas.getRegionLocations(RpcRetryingCallerWithReadReplicas.java:305)\n\tat org.apache.hadoop.hbase.client.ScannerCallableWithReplicas.call(ScannerCallableWithReplicas.java:156)\n\tat org.apache.hadoop.hbase.client.ScannerCallableWithReplicas.call(ScannerCallableWithReplicas.java:60)\n\tat org.apache.hadoop.hbase.client.RpcRetryingCaller.callWithoutRetries(RpcRetryingCaller.java:212)\n\tat org.apache.hadoop.hbase.client.ClientScanner.call(ClientScanner.java:314)\n\tat org.apache.hadoop.hbase.client.ClientScanner.loadCache(ClientScanner.java:432)\n\tat org.apache.hadoop.hbase.client.ClientScanner.next(ClientScanner.java:358)\n\tat org.apache.hadoop.hbase.client.AbstractClientScanner.next(AbstractClien